# Vertical GNN Model for Oral Bioavailability Dataset 

1. This notebook focuses on building the vertical model to predict oral bioavailability 
2. Hyperparameters were found using Optuna library which used the Tree-structured Parzen Estimator Algorithm in 30 trials. 
3. Model was trained/validated/tested using the best parameters and 5-fold CV. Whole process was repeated 10 times and the results were averaged. 

### Note
1. Ensure that this notebook is in the same working directory as data folder, config.py, utils.py, engine.py and model.py
2. To load models, please download the saved models provided in google drive link from README.md
3. Comment away the training function to load saved models and reproduce results

In [1]:
from utils import seed_everything, LoadHOBDataset
from config import SEED_NO, NUM_FEATURES, NUM_GRAPHS_PER_BATCH, NUM_TARGET, EDGE_DIM, DEVICE, PATIENCE, EPOCHS, N_SPLITS, params_vertical_gnn
from engine import EngineHOB
from model import VerticalGNN

import torch
import numpy as np
import optuna
from sklearn.model_selection import KFold
from torch_geometric.loader import DataLoader
import os 

## 2. Tuning the Model (Suggest to skip this step and go straight to the best parameters saved in config.py)

1. Model was tuned using Optuna library following the Tree-structured Parzen Estimator Algorithm for 30 trials. 
2. run_tuning function created to facilitate the train and validation step using an early stopping mechanism 
3. objective function created to facilitate the tuning process

In [5]:
def run_tuning(train_loader, valid_loader, params):
    model = VerticalGNN(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_gin_layers=params['num_gin_layers'], num_graph_trans_layers=params['num_graph_trans_layers'], 
                            hidden_size=params['hidden_size'], n_heads=params['n_heads'], dropout=params['dropout'], edge_dim=EDGE_DIM)
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineHOB(model, optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = PATIENCE
    early_stopping_counter = 0 

    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.validate(valid_loader)
        print(f'Epoch: {epoch+1}/{EPOCHS}, train loss : {train_loss}, validation loss : {valid_loss}')
        if valid_loss < best_loss:
            best_loss = valid_loss 
            early_stopping_counter=0

        else:
            early_stopping_counter +=1

        if early_stopping_counter > early_stopping_iter:
            print('Early stopping...')
            break
        print(f'Early stop counter: {early_stopping_counter}')
    
    return best_loss


In [6]:
def objective(trial):
    params = {
        'num_gin_layers' : trial.suggest_int('num_gin_layers', 1,3),
        'num_graph_trans_layers' : trial.suggest_int('num_graph_trans_layers', 1,3),
        'hidden_size' : trial.suggest_int('hidden_size', 64, 512),
        'n_heads' : trial.suggest_int('n_heads', 1, 5),
        'dropout': trial.suggest_float('dropout', 0.1,0.4),
        'learning_rate' : trial.suggest_float('learning_rate', 1e-3, 9e-3, log=True)
    }
    
    
    #load dataset 
    dataset_for_cv = LoadHOBDataset(root='./data/graph_data/data_oral_avail_train/', raw_filename='data_oral_avail_train_50.csv')
    kf = KFold(n_splits=N_SPLITS)
    fold_loss = 0

    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        print(f'Fold {fold_no}')
        train_dataset= []
        valid_dataset = []
        for t_idx in train_idx:
            train_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt'))
        for v_idx in valid_idx:
            valid_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt'))

        train_loader = DataLoader(train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)

        loss = run_tuning(train_loader, valid_loader, params)
        fold_loss += loss

    return fold_loss/5
if __name__ == '__main__':
    study = optuna.create_study(direction = 'minimize')
    study.optimize(objective, n_trials=30)
    print(f'best trial:')
    trial_ = study.best_trial
    print(trial_.values)
    print(f'Best parameters: {trial_.params}')

[I 2023-03-15 14:32:09,925] A new study created in memory with name: no-name-35b513d2-770b-4ff8-a043-0ce4ff834b9b


Fold 0
Epoch: 1/300, train loss : 6.748892426490784, validation loss : 3.036999464035034
Early stop counter: 0
Epoch: 2/300, train loss : 3.125407263636589, validation loss : 0.9028497934341431
Early stop counter: 0
Epoch: 3/300, train loss : 0.8272596895694733, validation loss : 0.7369771599769592
Early stop counter: 0
Epoch: 4/300, train loss : 0.7255954593420029, validation loss : 0.7451308369636536
Early stop counter: 1
Epoch: 5/300, train loss : 0.8327586799860001, validation loss : 0.6561927795410156
Early stop counter: 0
Epoch: 6/300, train loss : 0.6997271925210953, validation loss : 0.756048858165741
Early stop counter: 1
Epoch: 7/300, train loss : 0.6929575055837631, validation loss : 0.6666305661201477
Early stop counter: 2
Epoch: 8/300, train loss : 0.6753508299589157, validation loss : 0.7957857847213745
Early stop counter: 3
Epoch: 9/300, train loss : 0.717637374997139, validation loss : 0.6530423164367676
Early stop counter: 0
Epoch: 10/300, train loss : 0.69207571446895

[I 2023-03-15 14:32:44,340] Trial 0 finished with value: 0.6306962013244629 and parameters: {'num_gin_layers': 2, 'num_graph_trans_layers': 3, 'hidden_size': 371, 'n_heads': 1, 'dropout': 0.17015542288272026, 'learning_rate': 0.004964634551335803}. Best is trial 0 with value: 0.6306962013244629.


Epoch: 17/300, train loss : 0.6473052054643631, validation loss : 0.7519872188568115
Early stop counter: 10
Epoch: 18/300, train loss : 0.6421261727809906, validation loss : 0.7374377250671387
Early stopping...
Fold 0
Epoch: 1/300, train loss : 10.128466337919235, validation loss : 1.6040163040161133
Early stop counter: 0
Epoch: 2/300, train loss : 0.9656190276145935, validation loss : 0.6335681080818176
Early stop counter: 0
Epoch: 3/300, train loss : 0.6925968676805496, validation loss : 0.7112663984298706
Early stop counter: 1
Epoch: 4/300, train loss : 0.6760530322790146, validation loss : 0.6515933871269226
Early stop counter: 2
Epoch: 5/300, train loss : 0.7940814197063446, validation loss : 0.6253662109375
Early stop counter: 0
Epoch: 6/300, train loss : 0.693479061126709, validation loss : 0.654772937297821
Early stop counter: 1
Epoch: 7/300, train loss : 0.668770357966423, validation loss : 0.6547449231147766
Early stop counter: 2
Epoch: 8/300, train loss : 0.6681523472070694,

[I 2023-03-15 14:33:05,052] Trial 1 finished with value: 0.6325212240219116 and parameters: {'num_gin_layers': 3, 'num_graph_trans_layers': 1, 'hidden_size': 237, 'n_heads': 3, 'dropout': 0.309002774883653, 'learning_rate': 0.006668373458318256}. Best is trial 0 with value: 0.6306962013244629.


...

Epoch: 27/300, train loss : 0.6365452706813812, validation loss : 0.6806275844573975
Early stopping...
Fold 0
Epoch: 1/300, train loss : 0.7262941151857376, validation loss : 0.6819860935211182
Early stop counter: 0
Epoch: 2/300, train loss : 0.6951090842485428, validation loss : 0.7066640257835388
Early stop counter: 1
Epoch: 3/300, train loss : 0.6911783665418625, validation loss : 0.6842896938323975
Early stop counter: 2
Epoch: 4/300, train loss : 0.6887910664081573, validation loss : 0.697506844997406
Early stop counter: 3
Epoch: 5/300, train loss : 0.6846088469028473, validation loss : 0.6651571989059448
Early stop counter: 0
Epoch: 6/300, train loss : 0.6757830828428268, validation loss : 0.683417558670044
Early stop counter: 1
Epoch: 7/300, train loss : 0.6733037978410721, validation loss : 0.6313380599021912
Early stop counter: 0
Epoch: 8/300, train loss : 0.6815074384212494, validation loss : 0.6558529138565063
Early stop counter: 1
Epoch: 9/300, train loss : 0.667828619480133

[I 2023-03-15 14:44:13,560] Trial 28 finished with value: 0.6322495341300964 and parameters: {'num_gin_layers': 3, 'num_graph_trans_layers': 2, 'hidden_size': 69, 'n_heads': 1, 'dropout': 0.3199990661760711, 'learning_rate': 0.0043841732304825435}. Best is trial 22 with value: 0.6207164764404297.


Epoch: 16/300, train loss : 0.636401429772377, validation loss : 0.6701109409332275
Early stop counter: 9
Epoch: 17/300, train loss : 0.6319057196378708, validation loss : 0.7000323534011841
Early stop counter: 10
Epoch: 18/300, train loss : 0.6296630799770355, validation loss : 0.7435418367385864
Early stopping...
Fold 0
Epoch: 1/300, train loss : 0.7940010130405426, validation loss : 0.7035757899284363
Early stop counter: 0
Epoch: 2/300, train loss : 0.6939452290534973, validation loss : 0.6708582639694214
Early stop counter: 0
Epoch: 3/300, train loss : 0.6914562731981277, validation loss : 0.7078711986541748
Early stop counter: 1
Epoch: 4/300, train loss : 0.682930126786232, validation loss : 0.7248508930206299
Early stop counter: 2
Epoch: 5/300, train loss : 0.6960975825786591, validation loss : 0.6629201769828796
Early stop counter: 0
Epoch: 6/300, train loss : 0.6860993504524231, validation loss : 0.6317836046218872
Early stop counter: 0
Epoch: 7/300, train loss : 0.681071087718

[I 2023-03-15 14:44:29,899] Trial 29 finished with value: 0.6306508302688598 and parameters: {'num_gin_layers': 1, 'num_graph_trans_layers': 3, 'hidden_size': 117, 'n_heads': 1, 'dropout': 0.37418226724769776, 'learning_rate': 0.004902723569833367}. Best is trial 22 with value: 0.6207164764404297.


Epoch: 20/300, train loss : 0.6076437681913376, validation loss : 0.7799637317657471
Early stopping...
best trial:
[0.6207164764404297]
Best parameters: {'num_gin_layers': 2, 'num_graph_trans_layers': 2, 'hidden_size': 122, 'n_heads': 2, 'dropout': 0.36738054656589025, 'learning_rate': 0.00452976319043267}


## 3. Train/validate/test model

1. After tuning, the best parameters were saved in config.py as params_vertical_gnn
2. Model was trained/validated/tested using the best parameters and 5-fold CV. Process was repeated 5 times and the final results were averaged.
3. Ensure that data folder, from_scratch_trained_models folder and this notebook are in the same working directory
4. run_training to repeat the training process, please create a new folder to retrain, save and load saved model
5. Else, comment the run_training function and do testing to get the results which were obtained in the paper. 

In [2]:
def run_training(train_loader, valid_loader, params, trained_model_path):
    model = VerticalGNN(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_gin_layers=params['num_gin_layers'], num_graph_trans_layers=params['num_graph_trans_layers'], 
                            hidden_size=params['hidden_size'], n_heads=params['n_heads'], dropout=params['dropout'], edge_dim=EDGE_DIM)
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineHOB(model, optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = PATIENCE
    early_stopping_counter = 0 

    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.validate(valid_loader)
        print(f'Epoch: {epoch+1}/{EPOCHS}, train loss : {train_loss}, validation loss : {valid_loss}')
        if valid_loss < best_loss:
            best_loss = valid_loss 
            early_stopping_counter=0 #reset counter
            print('Saving model...')
            torch.save(model.state_dict(), trained_model_path)
        else:
            early_stopping_counter +=1

        if early_stopping_counter > early_stopping_iter:
            print('Early stopping...')
            break
        print(f'Early stop counter: {early_stopping_counter}')
    
    return best_loss

def run_testing(test_loader, params, trained_model_path):
    model = VerticalGNN(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_gin_layers=params['num_gin_layers'], num_graph_trans_layers=params['num_graph_trans_layers'], 
                            hidden_size=params['hidden_size'], n_heads=params['n_heads'], dropout=params['dropout'], edge_dim=EDGE_DIM)
    model.load_state_dict(torch.load(trained_model_path))
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineHOB(model, optimizer, device=DEVICE)

    print('Begin testing...')
    bce, acc, f1, roc_auc = eng.test(test_loader)
    print('Test completed!')
    print(f'bce:{bce}, acc:{acc}, f1: {f1}, roc_auc: {roc_auc}')
    return bce, acc, f1, roc_auc

In [4]:
params = params_vertical_gnn
n_repetitions = 5
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test/'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
path_to_save_trained_model = './from_scratch_trained_models/vertical/'

bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

#load dataset 
dataset_for_cv = LoadHOBDataset(root=train_data_root_path, raw_filename=train_data_raw_filename)
test_dataset = LoadHOBDataset(root=test_data_root_path, raw_filename=test_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions):
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset= []
        valid_dataset = []
        for t_idx in train_idx:
            train_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt'))
        for v_idx in valid_idx:
            valid_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt'))

        train_loader = DataLoader(train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)

        run_training(train_loader, valid_loader, params, os.path.join(path_to_save_trained_model, f'vertical_repeat_{repeat}_fold_{fold_no}.pt'))
        bce, acc, f1, roc_auc = run_testing(test_loader, params, os.path.join(path_to_save_trained_model, f'vertical_repeat_{repeat}_fold_{fold_no}.pt'))
        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

Epoch: 1/300, train loss : 0.9067558795213699, validation loss : 0.7183759808540344
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.7035864740610123, validation loss : 0.6651412844657898
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.6858863681554794, validation loss : 0.6628757119178772
Saving model...
Early stop counter: 0
Epoch: 4/300, train loss : 0.6888501644134521, validation loss : 0.6296036839485168
Saving model...
Early stop counter: 0
Epoch: 5/300, train loss : 0.6920813173055649, validation loss : 0.655631959438324
Early stop counter: 1
Epoch: 6/300, train loss : 0.6713947802782059, validation loss : 0.6383062601089478
Early stop counter: 2
Epoch: 7/300, train loss : 0.6747748851776123, validation loss : 0.6325588822364807
Early stop counter: 3
Epoch: 8/300, train loss : 0.6658817827701569, validation loss : 0.6485603451728821
Early stop counter: 4
Epoch: 9/300, train loss : 0.6621689051389694, validation loss : 0.6198813319206238
Savin